In [2]:
# librairies
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_score, classification_report, confusion_matrix, accuracy_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier, BallTree
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from IPython.display import display
from numpy import concatenate
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
import pandas as pd

In [4]:
#traitement fichier.mat
dataset=sp.loadmat('breastw.mat')
X = dataset['X']
y = dataset['y']
#fin traitement mat

#Variables pour test
liste_distances = ['euclidean', 'l1', 'l2', 'manhattan', 'matching', 
'minkowski', 'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'dice']
# Liste des métriques compatibles donnée par : print(sorted(BallTree.valid_metrics))
# METRIQUES QUI NE FONCTIONNENT PAS : 'pyfunc', 'seuclidean', 'wminkowski', 'infinity','p'
# fonctionnent mais retirés car trop de calculs : 'jaccard', 'kulsinski', 'hamming', 'rogerstanimoto', 'russellrao',  'sokalmichener', 'sokalsneath' , 

#Fin variables

Détermination des combinaisons des paramètres avec le meilleur F1-score

In [5]:
skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset

for train_index, test_index in skf.split(X,y):
    
    test =False
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np.ravel(y_train)
    y_test=np.ravel(y_test)

    test =False
    #for k in range(1,50,5) :
    for k in range(1,len(X_test),5) :
        for distance in liste_distances :
            #algo knn
            model_KNN = KNeighborsClassifier(n_neighbors=k, weights='distance', algorithm='auto', metric=distance)
            model_KNN.fit(X_train,y_train)
            y_pred_KNN = model_KNN.predict(X_test)
            
            F1_KNN = round(f1_score(y_test,y_pred_KNN),3)
            
            if test == False :
                matrice_KNN = np.array([[k,distance,F1_KNN]])
                test = True
            else :
                matrice_KNN = np.append(matrice_KNN,np.array([k,distance,F1_KNN]).reshape(1,3),axis=0)

df_KNN=pd.DataFrame(matrice_KNN,  columns=['Nombre_voisins', 'Distance','F1 score'])
df_KNN = df_KNN.astype({'F1 score': float})
filtered_values_1 = np.where((df_KNN['F1 score']>=0.98))
df_KNN = df_KNN.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False)
df_KNN = df_KNN.T

#display(df_KNN.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))
display(df_KNN)
#print(len(X_test))
#df_KNN.to_csv("Dataset comparaison param KNN.csv")

,17,66,71,28,73,83,84,94,95,105,...,140,129,107,41,118,96,52,85,74,151
Nombre_voisins,6,31,31,11,31,36,36,41,41,46,...,61,56,46,16,51,41,21,36,31,66
Distance,braycurtis,euclidean,minkowski,braycurtis,canberra,braycurtis,canberra,braycurtis,canberra,braycurtis,...,chebyshev,chebyshev,chebyshev,chebyshev,chebyshev,chebyshev,chebyshev,chebyshev,chebyshev,chebyshev
F1 score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98


Idem que précédent, avec les meilleurs F1-score, Recall, Precision, Accuracy et AUC

In [6]:
skf = StratifiedKFold(n_splits=10,shuffle=False) # équilibrage dataset

for train_index, test_index in skf.split(X,y):
    
    test =False
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np.ravel(y_train)
    y_test=np.ravel(y_test)

    test =False
    for k in range(1,len(X_test),5) : # pour le nombre de voisins allant de 1 au nombre de points dans X_test
        for distance in liste_distances : # et pour les distances 
            #algo knn
            model_KNN = KNeighborsClassifier(n_neighbors=k, weights='distance', algorithm='auto', metric=distance)
            model_KNN.fit(X_train,y_train)
            y_pred_KNN = model_KNN.predict(X_test)
            
            model_PCA = PCA(n_components=2)
            data_reduced = model_PCA.fit_transform(X_test)
            #fin algo knn
            y_scores_KNN = model_KNN.predict_proba(X_test)
            y_scores_KNN = y_scores_KNN[:, 1]
            
            F1_KNN = round(f1_score(y_test,y_pred_KNN),3)
            recall_KNN = round(recall_score(y_test,y_pred_KNN),3)
            precision_KNN = round(precision_score(y_test,y_pred_KNN,zero_division=0),3)
            accuracy_KNN = round(accuracy_score(y_test,y_pred_KNN),3)
            AUC_KNN = round(roc_auc_score(y_test, y_scores_KNN),3)
            
            if test == False :
                matrice_KNN = np.array([[k,distance,F1_KNN,recall_KNN,precision_KNN,accuracy_KNN,AUC_KNN]])
                test = True
            else :
                matrice_KNN = np.append(matrice_KNN,np.array([k,distance,F1_KNN,recall_KNN,precision_KNN,accuracy_KNN,AUC_KNN]).reshape(1,7),axis=0)

df_KNN=pd.DataFrame(matrice_KNN,  columns=['Nombre_voisins', 'Distance','F1 score','Recall','Precision','Accuracy','AUC'])
df_KNN = df_KNN.astype({'F1 score': float, 'Recall': float,'Precision': float,'Accuracy': float,'AUC': float})
filtered_values_1 = np.where((df_KNN['F1 score']>=0.975) & (df_KNN['Recall']>=0.90) & (df_KNN['Precision']>=0.95) & (df_KNN['AUC']>=0.95))
display(df_KNN.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))


,Nombre_voisins,Distance,F1 score,Recall,Precision,Accuracy,AUC
77,36,euclidean,1.000,1.000,1.0,1.000,1.000
111,51,l1,1.000,1.000,1.0,1.000,1.000
108,46,cityblock,1.000,1.000,1.0,1.000,1.000
104,46,minkowski,1.000,1.000,1.0,1.000,1.000
102,46,manhattan,1.000,1.000,1.0,1.000,1.000
...,...,...,...,...,...,...,...
6,1,braycurtis,0.979,0.958,1.0,0.985,0.979
5,1,minkowski,0.979,0.958,1.0,0.985,0.979
3,1,manhattan,0.979,0.958,1.0,0.985,0.979
2,1,l2,0.979,0.958,1.0,0.985,0.979
